<a href="https://colab.research.google.com/github/vsolodkyi/NeuralNetworks_SkillBox/blob/main/module_7/%D0%A0%D0%B5%D0%B0%D0%BB%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D1%8F_%D0%BC%D0%BE%D0%B4%D0%B5%D1%80%D0%BD%D0%B8%D0%B7%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%BD%D0%BE%D0%B9_FCN_%D1%81%D0%B5%D1%82%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Реализация модернизированной FCN сети
В этом уроке мы создадим модернизированную FCN модель с использованием изученных слоёв, которая потенциально должна выдвать более высокое качество на реальных задачах по семантической сегментации.

### Загрузка необходимых библиотек
Здесь мы загружаем разоичные библиотеки, включая TensoFlow.


In [ ]:
import numpy as np

import tensorflow as tf
# tf.enable_eager_execution()
print('TensorFlow version:', tf.__version__)

TensorFlow version: 2.2.0


### Создание FCN модели
Для создания модели создадим собственный класс, наследованный от `tf.keras.Model`.
Ранее мы использовали лишь свёртки и пулинги для извлечения признаков и понижения пространственных размерностий. Это был наш Энкодер. В качестве Декодера у нас был простой Upscale слой, который просто расширял изображение до нужного размера. Теперь же давайте реализуем более качественный Декодер с использованием слоёв для повышения разрешения.

В примере ниже будем использовать Unpooling (в керасе это делает слой `tf.keras.layers.UpSampling2D`) перемежающиийся со свёрточными слоями.

In [ ]:
class Model(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu')
        self.conv2 = tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu')        
        self.conv3 = tf.keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu')
        self.conv4 = tf.keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu')        
        self.conv5 = tf.keras.layers.Conv2D(256, (3, 3), padding='same', activation='relu')
        self.conv6 = tf.keras.layers.Conv2D(256, (3, 3), padding='same', activation='relu')    
        
        self.conv7 = tf.keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu')
        self.conv8 = tf.keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu')        
        self.conv9 = tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu')
        self.conv10 = tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu')
        self.conv11 = tf.keras.layers.Conv2D(1, (3, 3), padding='same', activation='sigmoid')
        
        self.pool = tf.keras.layers.MaxPool2D((2, 2))
        self.unpool = tf.keras.layers.UpSampling2D((2, 2))
                
    def call(self, x):
      
        # Encoder
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.pool(out)        
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.pool(out)        
        out = self.conv5(out)
        out = self.conv6(out)
        
        # Decoder        
        out = self.unpool(out)        
        out = self.conv7(out)
        out = self.conv8(out)
        out = self.unpool(out)        
        out = self.conv9(out)
        out = self.conv10(out)
        out = self.conv11(out)

        return out
    
model = Model()

### Задания


**[ЗАДАНИЕ 1]** Для такой модели для каждого промежуточного тензора с картами признаков укажите его размерности. Другими словами, для каждого вычисленного тензора `out` в каждой строчке функции `call()` выпишите его размерность, основываясь лишь на числах из конструктора. Размерность тензора `x` считать равной [1, 256, 256, 3]


**[ЗАДАНИЕ 2]** Создайте новую модель, аналогичную описанной выше, в котрой замените все Pooling слои на свёрточные слои со страйдом 2


**[ЗАДАНИЕ 3]** Реализуйте одну из написанных моделей (исходную или из Задания 2) с помощью функционального API кераса. Входом в модель сделайте узел `tf.keras.layers.Input((256, 256, 3))`. После этого визуализализируйте полученную модель с помощью `tf.keras.utils.plot_model(model, show_shapes=True)`. Проверьте размерности выходов всех слоёв, сравние с ответом из Задания 1.


**[ЗАДАНИЕ 4]** Реализуйте пайплайн обучения для такой модели (одной из): подготовка данных, лосс, обучение, тестирование. Используйте материалы из предыдущего практического урока. Обучите модель и сравните время обучения с временем обучения простой FCN из предыдущего урока.